# 全部采用本地化部署的方式和开源模型来建立的问答能力和本地化知识库集成的能力

定义语言模型

In [1]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True).half().cuda()
model = model.eval()

/home/xiaodong/anaconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No compiled kernel found.
Compiling kernels : /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.c -shared -o /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.so
Load kernel : /home/xiaodong/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 6
Using quantization cache
Applying quantization to glm layers


按照langchain标准进行模型封装

In [2]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any
from llama_index import LLMPredictor

class CustomLLM(LLM):

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response, history = model.chat(tokenizer, prompt, history=[])
        # only return newly generated tokens
        return response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": "THUDM/chatglm-6b-int4"}

    @property
    def _llm_type(self) -> str:
        return "custom"
    
llm_predictor = LLMPredictor(llm=CustomLLM())



引入embedding模型

In [3]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
))

定义包含embedding模型和语言模型的Service Context

In [4]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(embed_model=embed_model, llm_predictor=llm_predictor)

构建本地知识库（本地知识的索引库）

In [5]:
from langchain.text_splitter import CharacterTextSplitter
from llama_index import SimpleDirectoryReader
from llama_index import GPTVectorStoreIndex
from llama_index.node_parser import SimpleNodeParser

text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=20)
parser = SimpleNodeParser(text_splitter=text_splitter)
documents = SimpleDirectoryReader('./data/faq/').load_data()
nodes = parser.get_nodes_from_documents(documents)


index = GPTVectorStoreIndex(nodes=nodes, service_context=service_context)
query_engine = index.as_query_engine()

Created a chunk of size 130, which is longer than the specified 100


结合本地知识库和语言模型进行提问

In [6]:
question = """
请用中文回答下述问题：

Q: 你们能配送到天津吗？另外，你们的退货政策是怎样的？
"""
response = query_engine.query(question)
print(response)

The dtype of attention mask (torch.int64) is not bool


A: 我们可以配送到全国大部分省份，包括北京、上海、天津、重庆等。关于退货政策，我们在订单发货后的24小时内提供退货服务，如果符合退货条件，您可以在订单详情页查看退货流程。具体退货政策请以订单发货后的24小时内的页面信息为准。


In [7]:
question = """
请用中文回答下述问题：

Q: 请介绍一下ChatGLM？
"""
response = query_engine.query(question)
print(response)

ChatGLM-6B 是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型 GLM-6B 开发的，我的任务是针对用户的问题和要求提供适当的答复和支持。


In [8]:
question = """
请用中文回答下述问题：

Q: 请介绍一下语言模型中的Alpaca。
"""
response = query_engine.query(question)
print(response)

Alpaca是一个开源的语言模型，由清华大学 KEG 实验室和智谱AI开发。它被设计用来处理自然语言文本，并能够执行各种任务，如文本分类、命名实体识别、情感分析、机器翻译等。

Alpaca采用了深度学习技术，包括多层神经网络和卷积神经网络。它由多个预训练的模型组成，每个模型都针对特定的任务进行了训练。最终，这些预训练模型被组合在一起，形成了Alpaca。

Alpaca还采用了一些技术，如词向量、词嵌入、序列到序列模型等，以提高其在处理自然语言文本时的性能。它也能够与外部语言模型进行集成，如BERT、GPT等，以执行更加复杂的任务。

Alpaca是一个功能强大的语言模型，被广泛应用于自然语言处理领域。


In [9]:
question = """
请用中文回答下述问题：

Q: 请写一段python代码，展示如何使用Alpaca的模型在本地GPU运行支持语言任务的能力。
"""
response = query_engine.query(question)
print(response)

Token indices sequence length is longer than the specified maximum sequence length for this model (1258 > 1024). Running this sequence through the model will result in indexing errors


Alpaca 是一个基于深度学习的自然语言处理框架，可以在本地 GPU 上运行许多语言任务。下面是一段 Python 代码，展示如何使用 Alpaca 模型在本地 GPU 上运行支持语言任务的能力：
```python
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# Load the Alpaca model
model = load_model('alpaca_model.h5')

# Preprocess the text data
texts = [['这是一段文字', '这是一段文字']]
texts.insert(0, ['这是一段文字'])
texts.insert(0, ['这是一段文字'])
texts = [Tokenizer.text_to_sequences(text) for text in texts]

# Create a sequence of words for each text
words = [['这是一段文字', '这是一段文字']]
words.insert(0, ['这是一段文字'])
words = [Tokenizer.text_to_sequences(text) for text in words]

# Create a sequence of words for each text
words = [['这是一段文字', '这是一段文字']]
words.insert(0, ['这是一段文字'])
words = [Tokenizer.text_to_sequences(text) for text in words]

# Create a sequence of words for each text
words = [['这是一段文字', '这是一段文字']]
words.insert(0, ['这是一段文字'])
words = [Tokenizer.tex

使用新的TextSplitter来对原始内容进行处理，换一个新的index来处理索引，对文章进行总结

In [10]:
from langchain.text_splitter import SpacyTextSplitter

new_text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size = 2048)
new_parser = SimpleNodeParser(text_splitter=new_text_splitter)
new_documents = SimpleDirectoryReader('./data/mr_fujino').load_data()
new_nodes = new_parser.get_nodes_from_documents(new_documents)



In [11]:
from llama_index import GPTListIndex

list_index = GPTListIndex(nodes=new_nodes, service_context=service_context)

# response = list_index.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:", response_mode="tree_summarize")

query_engine = list_index.as_query_engine(
    response_mode="tree_summarize"
)
response = query_engine.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:")

print(response)

这段文字是鲁迅先生在回忆自己在中国学校的生活时写下的。藤野先生是自己的启蒙老师，让他去解剖实习和局部解剖学。他听说中国的女学生裹脚，但不知道具体细节，也感到有些为难。在学生会的一次活动中，他听说了中国人被枪毙的命运，被日本军捕获的中国学生也让他感到愤怒。在霉菌学课堂上，他观看了许多关于日俄战争的电影，其中有一些中国人被枪毙的场景让他感到刺耳。他后来改变了自己的态度，决定学习生物学，因为教授解剖学的先生给他留下了很好的印象。在离开仙台之前，他将自己的照片交给了藤野先生，并承诺会定期通信告诉他自己的状况。

整个文段表达了鲁迅对于一位重要老师的怀念之情，以及这位老师对自己人生的影响，同时也表达了自己对于学术和医学的追求和对于“正人君子”的反感。


换用TreeIndex来试试效果

In [12]:
from llama_index import GPTTreeIndex

# define LLM
tree_index = GPTTreeIndex(nodes=new_nodes, service_context=service_context)
# response = tree_index.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:", mode="summarize")

#query_engine = tree_index.as_query_engine(
#    response_mode="summarize"
#)

query_engine = tree_index.as_query_engine()

response = query_engine.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:")

print(response)

以下是新的总结：

藤野先生是一位留学生，在仙台的医学专门学校学习解剖学。他的学生很顽皮，藤野先生也会生气。藤野先生曾经批评学生的顽皮，并帮助学生改正错误。藤野先生喜欢中国的传统文化，也会讲述他在中国的经历。

在学年试验之后，鲁迅先生到东京玩了一夏天。秋初再回学校，成绩已发表，他在中间，没有落第。这回藤野先生所担任的功课是解剖实习和局部解剖学。

在解剖实习期间，藤野先生很高兴地叫我去，对我说：“我因为听说中国人是很敬重鬼的，所以很担心，怕你不肯解剖尸体。现在我总算放心了，没有这回事。”但他也偶有使我很为难的时候，听说中国的女人是裹脚的，但要问我怎么裹法，足骨变成怎样的畸形，还叹息道：“总要看一看才知道。

究竟是怎么一回事呢？”

有一天，本级的学生会干事到我寓里来，要借我的讲义看，我检出来交给他们，却只翻检了一通，并没有带走。但他们一走，邮差就送到一封很厚的信，打开一看，第一句是：“你改悔罢！”这是《新约》上的句子，但经托尔斯泰新近引用过。


尝试Vector Store - pinecone

In [13]:
'''
# 目前这个不分不工作
import pinecone
from llama_index import StorageContext
from llama_index.vector_stores import PineconeVectorStore
import os

# pinecone_api_key = os.environ.get("PINECONE_API_KEY")
# pinecone_env = os.environ.get("PINECONE_ENV")

pinecone_api_key = os.environ['PINECONE_API_KEY']
pinecone_env = os.environ['PINECONE_ENV']


# init pinecone
pinecone.init(pinecone_api_key, pinecone_env)
pinecone.create_index("xdtest1", dimension=1536, metric="cosine", pod_type="p1") # pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")

# construct vector store and customize storage context
storage_context = StorageContext.from_defaults(
    vector_store = PineconeVectorStore(pinecone.Index("xdtest1"))
)

# Load documents and build index
documents_p = SimpleDirectoryReader('./data/mr_fujino').load_data()
index_p = GPTVectorStoreIndex.from_documents(documents_p, service_context=service_context, storage_context=storage_context)

query_engine = index_p.as_query_engine()

response = query_engine.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:")

print(response)
'''

'\n# 目前这个不分不工作\nimport pinecone\nfrom llama_index import StorageContext\nfrom llama_index.vector_stores import PineconeVectorStore\nimport os\n\n# pinecone_api_key = os.environ.get("PINECONE_API_KEY")\n# pinecone_env = os.environ.get("PINECONE_ENV")\n\npinecone_api_key = os.environ[\'PINECONE_API_KEY\']\npinecone_env = os.environ[\'PINECONE_ENV\']\n\n\n# init pinecone\npinecone.init(pinecone_api_key, pinecone_env)\npinecone.create_index("xdtest1", dimension=1536, metric="cosine", pod_type="p1") # pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")\n\n# construct vector store and customize storage context\nstorage_context = StorageContext.from_defaults(\n    vector_store = PineconeVectorStore(pinecone.Index("xdtest1"))\n)\n\n# Load documents and build index\ndocuments_p = SimpleDirectoryReader(\'./data/mr_fujino\').load_data()\nindex_p = GPTVectorStoreIndex.from_documents(documents_p, service_context=service_context, storage_context=storage_context)\n\

Prompt template的支持

In [16]:
from llama_index import QuestionAnswerPrompt

# load documents
new_documents = SimpleDirectoryReader('./data/faq/').load_data()

# define custom QuestionAnswerPrompt
query_str = "请问你们海南能发货吗？"
QA_PROMPT_TMPL = (
    "请参考如下上下文信息. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "基于这样的上下文信息，请回答: {query_str}\n"
)
QA_PROMPT = QuestionAnswerPrompt(QA_PROMPT_TMPL)
# Build GPTVectorStoreIndex
index = GPTVectorStoreIndex.from_documents(new_documents, service_context=service_context)

query_engine = index.as_query_engine(
    text_qa_template=QA_PROMPT
)
response = query_engine.query(query_str)
print(response)

我们支持海南地区的发货。您可以在“我的订单”页面查看物流信息，并选择支持海南地区的物流公司进行发货。
